In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_380.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_517.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_364.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_258.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_237.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_476.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_563.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_159.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_464.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_518.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_141.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_398.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_442.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_443.jpg
/kaggl

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = "/kaggle/input/cats-and-dogs-image-classification/train"
test_dir = "/kaggle/input/cats-and-dogs-image-classification/test"

# Load datasets
batch_size = 32
img_size = (224, 224)

# Load dataset
train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=img_size,
    validation_split=0.2,
    subset="training",
    seed=42
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=img_size,
    validation_split=0.2,
    subset="validation",
    seed=42
)

Found 557 files belonging to 2 classes.
Using 446 files for training.
Found 140 files belonging to 2 classes.
Using 28 files for validation.


In [17]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# Optimize dataset loading for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


In [18]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2

# Load pretrained MobileNetV2 (without top layers)
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

# Freeze base model layers (so they don't get retrained)
base_model.trainable = False

# Build model
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),  # Reduce overfitting
    layers.Dense(1, activation="sigmoid")  # Binary classification (Cat/Dog)
])

# Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [19]:
epochs = 10
history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)


Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 17s 834ms/step - accuracy: 0.8120 - loss: 0.4022 - val_accuracy: 1.0000 - val_loss: 0.0613
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 725ms/step - accuracy: 0.9709 - loss: 0.0697 - val_accuracy: 1.0000 - val_loss: 0.0254
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 728ms/step - accuracy: 0.9980 - loss: 0.0231 - val_accuracy: 1.0000 - val_loss: 0.0123
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 742ms/step - accuracy: 0.9939 - loss: 0.0212 - val_accuracy: 1.0000 - val_loss: 0.0245
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 698ms/step - accuracy: 1.0000 - loss: 0.0106 - val_accuracy: 1.0000 - val_loss: 0.0122
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 729ms/step - accuracy: 0.9978 - loss: 0.0070 - val_accuracy: 1.0000 - val_loss: 0.0151
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 766ms/step - accuracy: 0.9969 - loss: 0.0095 - val_accuracy: 1.0000 - val_loss: 0.0123
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 779ms/step - accuracy: 1.0000 - loss: 0.0040 - val_accu

In [20]:
# Evaluate performance
loss, accuracy = model.evaluate(test_dataset)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Predict a single image
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_380.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
print("It's a Dog!" if prediction[0] > 0.5 else "It's a Cat!")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 661ms/step - accuracy: 1.0000 - loss: 0.0105
Validation Accuracy: 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
It's a Dog!


In [21]:
import os
from tensorflow.keras.preprocessing import image

test_folder = "/kaggle/input/cats-and-dogs-image-classification/test/cats"
image_files = [os.path.join(test_folder, img) for img in os.listdir(test_folder) if img.endswith(".jpg")]

for img_path in image_files:
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    label = "Dog" if prediction[0] > 0.5 else "Cat"
    
    print(f"{img_path} → {label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
/kaggle/input/cats-and-dogs-image-classification/test/cats/cat_585.jpg → Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
/kaggle/input/cats-and-dogs-image-classification/test/cats/cat_595.jpg → Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
/kaggle/input/cats-and-dogs-image-classification/test/cats/cat_525.jpg → Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
/kaggle/input/cats-and-dogs-image-classification/test/cats/cat_306.jpg → Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
/kaggle/input/cats-and-dogs-image-classification/test/cats/cat_430.jpg → Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
/kaggle/input/cats-and-dogs-image-classification/test/cats/cat_551.jpg → Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
/kaggle/input/cats-and-dogs-image-classification/test/cats/cat_332.jpg → Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
/kaggle/input/cats-and-dogs-image-classification/test/cats/cat_156.jpg → Cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
/kaggle/input/cats-and-dogs-image-classifi

In [22]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.2%}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step - accuracy: 1.0000 - loss: 0.0105
Test Accuracy: 100.00%
